OptionGun03_Tonka
Notebook created 12/07/22
12/27/22 - Tested and running

User Notes
1. This program needs a user key from Darqube.com, which it uses to get real-time stock prices.
   Go to the Darqube website get a user account and choose the free service. Then get your key.
   Find the Dar_key line just below the "User Parameters" block in the cell below.
   Enter it between the quotes on the line:  DAR_key = ''
   
2. OptionGun produces three csv files:
     root_data.csv. This contains a list of all tickers, with real-time price at the time of execution, 
     and the netx Earnings Report date for each ticker, if available. DO NOT CHANGE THIS FILE.
     all_optinos_suffix.csv. This file contains every option available for every ticker in the list. 
     best_options_suffix.csv. This file contains only the options with meet the selection criteria in the program.
     
2. The parameters just under the "User Parameters" header block can be changed.
    Add and remove ticker symbols from the TICKERS = list. Enclose each ticker Follow the format shown.
    HIDE_TICKERS. You may want to collect option info for a ticker, but not see that ticker in best_file.csv.
    Just put the ticker to be hidden in this list. 
    FILE_ID_TAG is the suffix that OptionGun will attach to all_options and best_options. You can change it at any time.
    
3. OptionGun needs about 2-3 seconds to get the option string for each ticker. For 100 tickers, it will take about 5 minutes.

OptionGun Structure

- HEADERS and CONSTANTS

- Broker_Converter_Van - TEMPORARY here to get Vanguard report and produce trade_log_open for BTC offers.

- GET OPTION DATA
       - Get real-time price data from Darqube.
       - Get option strings from yfinance.
       - Get ER dates from yfinance.
       - Parse option strings into common POWMax format.

Future
- Put_STO_Gun - Screen and select PSTO offers
- Call_STO_Gun- Screen and select CSTO offers
- Put_BTC_Gun - Screen and select PBTC offers
- Call_BTC_Gun- Screen and select CBTC offers

TODO 12/13/22
1. Use new column lables to split broker_report into three.
2. Delete fill-col headers. Replace with correct column headers.

DONE 12/23/22
12/23/23 - Added best_STO_LEAP. 
12/23/22 - Added XLOptionBoard to load best_STO and best_STO_LEAP


In [1]:
# ****************
#
#Header and Constants
#
# *****************
# Purpose - OptionGun downlaods option strings and ER dates from yahoo yfinance.
#                It downloads real-time stock prices and company fundamental financial data from Darqube.
#                It calculates metrics for options 
#                and selects options that score the best on these metrics: POW, ARR, PctOTM, and Expiry DaysOut.
#
# Started 11/11/19 from TQIRobot.03 with yfinance library
#
#   12/05/22 - Forked version for Tonka_Prdtn.
#   12/05/22 - Added tickers with high Quant rating. Tightened limits for best_options.
#   11/27/22 - Remove IEX. Now using Darqube for market data. Added many more tickers, approx. 150 total.
#
# *****************
#
# USER PARAMS
# User can change these parameters as appropriate.
#
# *****************

DAR_key = '' # Get one from Darqube and enter it here, between the single quotes.

TICKERS = ('URI', 'JEF', 'BSM', 'BGSF', 'CPA', 'STLD', 'AMR', 'UNVR', 'CVCO', 'HUBG', 'ACLS', 'ASRT', \
           'AFYA', 'SMCI', 'HDSN', 'VIST', 'VRNA', 'NWG', 'PINE', 'VIR', 'ASO', 'PERI', 'STRL', 'VPG', \
           'AAPL', 'ABBV', 'AFL', 'ADBE', 'AIG', 'AMD', 'AMT', 'AMZN', 'AOS', 'ARNC', 'AVB', 'AVGO', \
           'AZO', 'BA', 'BAC', 'BBBY', 'BIIB', 'BNTX', 'CAG', 'CAT', 'CCI', 'CF', 'CL', \
           'CLX', 'CMCSA', 'CRM', 'CRWD', 'CSCO', 'CSX', 'CVS', 'CVX', 'DAC', 'DDOG', \
           'DHI', 'DUK', 'DVN', 'EPAM', 'EPD', 'EPR', 'ET', 'EXPE', 'FAF', 'FANG', 'FDX', 'FLNG', \
           'FMS', 'FNF', 'FSLY', 'FSR', 'FYBR', 'GBX', 'GD', 'GIS', 'GLNG', 'GOOGL', 'GS', 'GSL', 'HD', 'HON', \
           'HR', 'HRB', 'IBM', 'INTC', 'IPI', 'IRM', 'JNJ', 'JPM', 'K', 'KO', 'LADR', 'LMT', 'LOW', \
           'LRCX', 'LYB', 'MA', 'MAN', 'MCK', 'META', 'MGA', 'MLM', 'MMM', 'MMP', 'MOS', 'MPW', \
           'MRK', 'MRNA', 'MRVL', 'MSFT', 'MU', 'NFLX', 'NKE', 'NKTR', 'NMM', 'NOW', 'NRG', 'NVDA', \
           'OKTA', 'ON', 'PANW', 'PFE', 'PHM', 'PII', 'PLTR', 'PRU', 'PYPL', 'QCOM', 'RITM', \
           'RIVN', 'ROST', 'RTX', 'SB', 'SBLK', 'SBUX', 'SCHW', 'SKT', 'SNOW', 'SO', 'SPG', 'SPY', \
           'STOR', 'STWD', 'TSLA', 'TRV', 'TXN', 'TSCO', 'TSN', 'UNP', 'UPS', 'USAC', 'V', 'VICI', 'VGT', 'VMW', \
           'VNOM', 'VOO', 'WEC', 'WMT', 'WSM', 'WFC', 'XOM', 'ZIM', 'ZS')

#TICKERS = ('AMZN', 'MSFT', 'XOM', 'NFLX', 'SPY')

HIDE_TICKERS = ('AOS', 'EPAM', 'FSLY', 'NOW', 'PLTR', 'VNOM', 'RIVN', 'NKTR') # Keep these tickers for database, but do not show them in best_options, because you do not want to own them.  

FILE_ID_TAG       = '_Tonka'

# *****************
#
# Import Libraries
#
# *****************

import csv
import copy
import json
import numpy as np
import os
import pandas as pd
import requests   # for http requests
import scipy
from   scipy import stats
from   scipy.stats import norm
import time
import yfinance as yf

import datetime
from   datetime import datetime
from   datetime import date
from   datetime import timedelta
#from   datetime import fromtimestamp

from   dateutil.relativedelta  import relativedelta
from   yahoo_earnings_calendar import YahooEarningsCalendar

#import investpy
#import glob
#import collections
#from enum import Enum
#import re     # re is regular expression. For searching strings.

# *****************
#
# Flow Control
# Internal Constants
#
# *****************

update_root_data      = True  #skip WTI call if testing code. Just use synthetic WTI_dict instead.
update_root_data_ER   = True  #skip call to yahoo calendar if testing code.
update_option_data    = True  #skip calls to yfinance if testing code.


profile = 'POWARRMax'

DATE_FORMAT       = '%Y-%m-%d %H:%M:%S'

TICKER_DELAY      = 0.020  # seconds between tickers for data requests.
EXPIRY_DELAY      = 0.005

MAX_DAYSOUT = 366.0  # This constant controls how many expiry strings are downloaded. Use low # to exclude BTOS and reduce ext. calls. Hi # includes BTOS.
MIN_DAYSOUT = 3.0

FEE_SPREAD = 0.75     # Fee in the bid/ask range.

call_ticker_options = pd.DataFrame()
put_ticker_options  = pd.DataFrame()
ticker_options      = pd.DataFrame() # for a single ticker
all_options         = pd.DataFrame() # includes options for all tickers. Do not use any limits or exclusions for all_options.
good_options        = pd.DataFrame() # Has rankings for CSTO options and BTOs (CBTO).
stats               = pd.DataFrame() # Not used now.
options_open        = pd.DataFrame() # persistent file used by OptionCloser to recommend BTC offers.
    


In [2]:
# *****************
# Get option data.
#
# 1. Get real-time price data from Darqube. Write to root_data. 
# 2. Get ER dates from Yahoo Earnings Calendar and write to root_data.
# 3. Get option strings from yfinance. Use "try" statements to skip over exp dates where call to yfinance failed.
# 4. Parse the option data fields into the common POWMax formats. 
# 
# *****************

#Use Darqube for stock price data - both market and historical.

root_data = pd.read_csv('root_data.csv')
root_data.set_index('Ticker', inplace = True, drop = True)

print('start Darqube access at:  ', datetime.now(), '  Done in ',len(TICKERS) * 1.0 / 60.0, ' minutes.')
for ticker in TICKERS:

    price_rqst = 'https://api.darqube.com/data-api/market_data/quote/' + ticker + '?token=' + DAR_key
    response = requests.get(price_rqst)
    price_dict = response.json()
    
    root_data.loc[ticker, 'root price'] = price_dict['price']

root_data.sort_index(inplace = True) #Save root_data with updated ER dates.
root_data.to_csv('root_data.csv')
print(datetime.now())

if update_root_data_ER:                # skip call to YEC for ER update
    days_to_check_ER = (1, 4)  #check for ER updates when day # matches any in this list. This combo guarantees ER check on at least one date each month. 
    print('today date = ', date.today().day)
    if date.today().day in days_to_check_ER:
        yec_custom_delay_s = 0.1
        yec = YahooEarningsCalendar(yec_custom_delay_s)
        print('      Just updating ER dates.')

        for ticker in TICKERS:
            try:
                next_er_date = (datetime.fromtimestamp(yec.get_next_earnings_date(ticker)).strftime('%Y-%m-%d %H:%M'))
            except: 
                print(ticker, ' got retrieval error')
            else:
                root_data.loc[ticker, 'ER Date'] = next_er_date
                print(ticker, root_data.loc[ticker, 'ER Date'])
        root_data.sort_index(inplace = True) #Save root_data with updated ER dates.
        root_data.to_csv('root_data.csv')

datetoday = date.today()
#root_data['ER daysout'] = np.where(root_data['ER Date'] != None,(root_data['ER Date'] - pdatetoday).datetime.days, NaN) # convert these to date code to subtract

root_data.sort_index(inplace = True)
root_data.to_csv('root_data.csv')

if update_option_data == False: # then skip the actual data collection from the net. Just grab the all_options file and calculate good_options.
#    read_file = 'all_options' + FILE_ID_TAG + '.csv'
    read_file = 'all_options' + '2022-03-22' + '.csv'
    all_options = pd.read_csv(read_file)
    
    print('skipped yahoo options update.')
else:
    for ticker in TICKERS:
        print('getting data for ', ticker, ' at time ', datetime.now())
        try:                                      # get option strings from yfinance for each ticker.
                                                  # if call to yfinance fails, then just skip this expiry date.
            ticker_data = yf.Ticker(ticker)       # get option strings from yfinance for each ticker.
        except:
            print('yfinance call failed for ', ticker) 
        quote_time = datetime.now() + timedelta(hours = 3)
        quote_date = date.today()
        expiry_dates = ticker_data.options
        call_ticker_options = pd.DataFrame()
        put_ticker_options  = pd.DataFrame()
        for exp_date in expiry_dates:
            exp_daysout = (datetime.strptime(exp_date, '%Y-%m-%d').date() - quote_date).days
            if (exp_daysout < MAX_DAYSOUT and exp_daysout > MIN_DAYSOUT):
                #if exp_date != '2020-04-02': #test code only. For normal operation delete this.
                    
                    #If doing BTOs set MAX_DAYSOUT to high number, say 1000.0.
                    try:
                        option_data = ticker_data.option_chain(exp_date)
                        call_ticker_options = pd.concat([call_ticker_options, option_data.calls], sort = False, ignore_index = True)
                        put_ticker_options  = pd.concat([put_ticker_options,  option_data.puts],  sort = False, ignore_index = True)
                    except:
                        print('     yfinance call failed for expiry ', exp_date)
                    time.sleep(EXPIRY_DELAY)           
        call_ticker_options['option_type'] = 'call'
        put_ticker_options['option_type'] = 'put'

        # calculate columns unique to each ticker        
        ticker_options = pd.concat([call_ticker_options, put_ticker_options], sort = False, ignore_index = True)
        ticker_options['root'] =  ticker
        ticker_options['root price'] = root_data.loc[ticker, 'root price']
        ticker_options['Quote_Time'] = quote_time
        ticker_options['ER Date'] = root_data.loc[ticker, 'ER Date']
        ticker_options['Expstring'] = '20' + ticker_options['contractSymbol'].str[len(ticker):len(ticker) + 6]
        ticker_options['Expiry'] = ticker_options['Expstring'].str[:4] + '-' + ticker_options['Expstring'].str[4:6] + '-' + \
                                   ticker_options['Expstring'].str[6:8]
        all_options = pd.concat([all_options, ticker_options], sort = False, ignore_index = True)

        time.sleep(TICKER_DELAY)

    print('Got all the data for all tickers. Created additional cols. time = ', datetime.now())


start Darqube access at:   2022-12-27 16:00:32.341879   Done in  0.08333333333333333  minutes.
2022-12-27 16:00:36.445876
today date =  27
getting data for  AMZN  at time  2022-12-27 16:00:36.445876
getting data for  MSFT  at time  2022-12-27 16:00:40.064300
getting data for  XOM  at time  2022-12-27 16:00:43.300647
getting data for  NFLX  at time  2022-12-27 16:00:45.944864
getting data for  SPY  at time  2022-12-27 16:00:49.244784
Got all the data for all tickers. Created additional cols. time =  2022-12-27 16:00:54.557455


In [3]:
# ****************
#
#Cell 4 - Calculate metrics for each option.
#
# *****************
all_options['Quote_Time'] = pd.to_datetime(all_options['Quote_Time'], format = DATE_FORMAT)
all_options['Expiry']     = pd.to_datetime(all_options['Expiry'], format = DATE_FORMAT)
all_options['fee']        = FEE_SPREAD * (all_options['ask'] - all_options['bid']) + all_options['bid']
all_options['daysout']    = (all_options['Expiry'] - all_options['Quote_Time']) / np.timedelta64(1,'D')
all_options['strike']     = pd.to_numeric(all_options['strike'], errors = 'coerce')
all_options['root price'] = pd.to_numeric(all_options['root price'], errors = 'coerce')
#all_options['ER Date']    = pd.to_datetime(all_options['ER Date'], format = DATE_FORMAT)
#all_options['ER daysout'] = (all_options['ER Date'] - all_options['Quote_Time']) / np.timedelta64(1,'D')
#all_options['ER-Expiry']  = all_options['ER daysout'] - all_options['daysout']

all_options['OTM']     = np.where(all_options['option_type'] == 'call', all_options['strike'] - all_options['root price'], \
                                  all_options['root price'] - all_options['strike'])

# use np.where to calculate different ARRs based on put/call and buy/sell
all_options['ARR']     = 100.0 * (all_options['fee'] * 365) / (all_options['daysout'] * all_options['root price'])
all_options['PctOTM']  = 100.0 * all_options['OTM'] / all_options['root price']

#all_options['TQI']     = all_options['ARR'] * all_options['PctOTM'] /10

all_options['callPITM'] = 100.0 * norm.cdf(np.log(all_options['root price'] / all_options['strike']) /
                                  (all_options['impliedVolatility'] * (all_options['daysout'] / 365)**(1/2)))

all_options['POW'] = np.where(all_options['option_type'] == 'call', 100.0 - all_options['callPITM'], all_options['callPITM'])

all_options['PctFee'] = 100.0 * all_options['fee'] / all_options['root price']
all_options['BidAskSpread'] = 100.0 * (all_options['ask'] - all_options['bid']) / all_options['bid']
all_options['impliedVolatility'] = 100.0 * all_options['impliedVolatility']

write_file = 'all_options' + FILE_ID_TAG + '.csv'
all_options.to_csv(write_file, index = False)
print('Wrote all_options')



Wrote all_options


In [4]:
#****************
# Select best options based on selected profile. Currently the only profile allowed is 
#*****************

best_STO = all_options.copy()


# best_STO uses POWARRMax profile
POWmin     = 69
POWmax     = 85
ARRmin     = 19.
ARRmax     = 95.
PctOTMmin  =  4.0
PctOTMmax  = 18.0
PctFeemin  =  2.0
PctFeemax  = 12.0
daysoutmax = 55.0
daysoutmin =  2.0


best_STO = best_STO[(((best_STO['POW']    >= POWmin)    & (best_STO['POW']    <= POWmax))    & 
                     ((best_STO['ARR']    >= ARRmin)    & (best_STO['ARR']    <= ARRmax)))   &
                    (((best_STO['PctFee'] >= PctFeemin) & (best_STO['PctFee'] <= PctFeemax)) & 
                     ((best_STO['PctOTM'] >= PctOTMmin) & (best_STO['PctOTM']  <= PctOTMmax)))].copy()

best_STO = best_STO[(best_STO['daysout']  >= daysoutmin) & (best_STO['daysout'] <= daysoutmax)].copy() 

best_STO = best_STO[~best_STO['root'].isin(HIDE_TICKERS)].copy()

#convert some columns to int

int_columns = ['daysout', 'ARR', 'POW', 'impliedVolatility']
best_STO[int_columns] = best_STO[int_columns].astype(int)
                           
# remove temporary columns in best_STO
drop_columns = ['contractSymbol', 'inTheMoney', 'lastTradeDate', 'lastPrice', 'bid', 'ask', \
                'change', 'percentChange', 'callPITM', 'BidAskSpread', \
                'volume', 'openInterest', 'Quote_Time', 'Expstring', 'contractSize', 'currency']

#Removed ER daysout and ER-Expiry from drop_columns. Putthe back after fixing the format problem.

best_STO.drop(labels = drop_columns, axis = 1, inplace = True)

#Sort your favorite way

best_STO.sort_values(['root', 'option_type', 'POW'], ascending = [True, True, False], inplace = True)

write_file = 'best_STO' + FILE_ID_TAG + '.csv'
best_STO.to_csv(write_file, index = False, float_format = '%.2f')


print('Wrote best_STO. Finished at ', datetime.now())

# Calculate best_STO_LEAP with LEAP Profile in fct below.
# Use this fct to run all screens on all_options

def run_best_STO(all_options):
    # Set up this function to run with any screen.
    # LEAP Profile
    POWmin     = 5
    POWmax     = 95
    ARRmin     = 5.0
    ARRmax     = 500.0
    PctOTMmin  = -100.0
    PctOTMmax  =  100.0
    PctFeemin  =    5.0
    PctFeemax  =  100.0
    daysoutmin =  175.0
    daysoutmax = 1000.0

    best_STO_LEAP = all_options.copy()

    best_STO_LEAP = best_STO_LEAP[(((best_STO_LEAP['POW']    >= POWmin)    & (best_STO_LEAP['POW']    <= POWmax))    &  
                                   ((best_STO_LEAP['ARR']    >= ARRmin)    & (best_STO_LEAP['ARR']    <= ARRmax)))   & 
                                  (((best_STO_LEAP['PctFee'] >= PctFeemin) & (best_STO_LEAP['PctFee'] <= PctFeemax)) & 
                                   ((best_STO_LEAP['PctOTM'] >= PctOTMmin) & (best_STO_LEAP['PctOTM'] <= PctOTMmax)))].copy()

    best_STO_LEAP = best_STO[(best_STO_LEAP['daysout']  >= daysoutmin) & (best_STO_LEAP['daysout'] <= daysoutmax)].copy() 

    best_STO_LEAP = best_STO_LEAP[~best_STO['root'].isin(HIDE_TICKERS)].copy()

    #convert some columns to int

    int_columns = ['daysout', 'ARR', 'POW', 'impliedVolatility']
    best_STO_LEAP[int_columns] = best_STO_LEAP[int_columns].astype(int)
                           
    # remove temporary columns in best_STO
    drop_columns = ['contractSymbol', 'inTheMoney', 'lastTradeDate', 'lastPrice', 'bid', 'ask', \
                    'change', 'percentChange', 'callPITM', 'BidAskSpread', \
                    'volume', 'openInterest', 'Quote_Time', 'Expstring', 'contractSize', 'currency']

    #Removed ER daysout and ER-Expiry from drop_columns. Putthe back after fixing the format problem.

    best_STO_LEAP.drop(labels = drop_columns, axis = 1, inplace = True)

    #Sort your favorite way

    best_STO_LEAP.sort_values(['root', 'option_type', 'POW'], ascending = [True, True, False], inplace = True)

    write_file = 'best_STO_LEAP' + FILE_ID_TAG + '.csv'
    best_STO_LEAP.to_csv(write_file, index = False, float_format = '%.2f')

    print('wrote best_STO_LEAP')
    
    return best_STO_LEAP

LEAP = False
if LEAP:
    best_STO_LEAP = best_STO(all_options)


Wrote best_STO. Finished at  2022-12-27 16:00:54.994692
